In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

# fp_df_eval_results = 'df_eval_results'
# fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.

###################################  USE df_val as df_train
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
df_train = df_val.copy()
###################################  USE df_val as df_train

len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 300, len_df_val: 300, len_df_test: 150 


In [3]:
print(f'df_train.index: {df_train.index[0]}, {df_train.index[-1]}')
print(f'df_val.index:   {df_val.index[0]}, {df_val.index[-1]}')

df_train.index: 2021-05-13 00:00:00, 2022-07-21 00:00:00
df_val.index:   2021-05-13 00:00:00, 2022-07-21 00:00:00


In [4]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


In [5]:
# # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_validate)
# df

In [ ]:
# if store_results:  # record results to df
# my_cols = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'SPY_CAGR/UI']
# df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
df_eval_sym_freq_results_validate = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
df_eval_sym_freq_results_validate

In [294]:
# verbose = True  # True prints more output
verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
n_samples = 114  
# n_samples = 1

# for training, the number of days to lookback from iloc max-lookback end_train
days_lookbacks = [15, 30, 60, 120]
days_lookbacks = [30, 60, 120]
days_lookbacks = [60, 120]
days_lookbacks = [120]
days_lookbacks = [60]
days_lookbacks = [30]
days_lookbacks = [15]
days_lookbacks = [15, 30]
days_lookbacks = [15, 30, 60]


days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
# days_eval = 10
days_eval = 6
days_eval = 5
days_eval = 4
# days_eval = 3
# days_eval = 2  


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
# syms_start = 1  #  start index of n_top_syms for evaluation

# syms_end = n_top_syms  #  end index of n_top_syms for evaluation
# syms_end = 1  #  end index of n_top_syms for evaluation
# syms_end = 2  #  end index of n_top_syms for evaluation
# syms_end = 3  #  end index of n_top_syms for evaluation
# syms_end = 4  #  end index of n_top_syms for evaluation
# syms_end = 5  #  end index of n_top_syms for evaluation
# syms_end = 6  #  end index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [295]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [296]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    _df = df_train.iloc[start_train:end_train]
    date_start_train = _df.index[0].strftime('%Y-%m-%d')
    date_end_train = _df.index[-1].strftime('%Y-%m-%d')

    if verbose:
      print(f'days lookback:    {lookback}')
      print(f'lb_slices:        {lb_slices}')
      print(f'lb_slice:         {lb_slice}')
      print(f'days eval:        {d_eval}')    
      print(f'iloc_start_train: {start_train}')
      print(f'iloc_end_train:   {end_train}')
      print(f'date_start_train: {date_start_train}')
      print(f'date_end_train:   {date_end_train}')


    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

In [297]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    # if verbose:
    #     print(f'sym_freq_cnt_6: {sym_freq_cnt_6}')
    #     print(f'sym_freq_cnt_5: {sym_freq_cnt_5}')
    #     print(f'sym_freq_cnt_4: {sym_freq_cnt_4}')
    #     print(f'sym_freq_cnt_3: {sym_freq_cnt_3}')
    #     print(f'sym_freq_cnt_2: {sym_freq_cnt_2}')

    # return sym_freq_cnt_6, sym_freq_cnt_5, sym_freq_cnt_4, sym_freq_cnt_3, sym_freq_cnt_2
    return l_sym_freq_cnt    

In [298]:
for top_set_syms_n_freq in grp_top_set_syms_n_freq:
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}')
  print(f'sym_freq_2: {l_sym_freq_cnt[4]}\n')  

sym_freq_6: ['AMPH', 'EQT', 'GLNG', 'LNTH']
sym_freq_5: ['BPT', 'SBS']
sym_freq_4: ['CALM', 'IPI', 'OXY', 'RGLD']
sym_freq_3: ['AR', 'CIB', 'GOGO', 'IRTC', 'MOS', 'PTEN', 'RGNX', 'RRC', 'SPLK']
sym_freq_2: ['CUTR']

sym_freq_6: ['DDS', 'ENPH', 'PDFS', 'WIRE', 'WOLF']
sym_freq_5: ['ANET', 'WD']
sym_freq_4: ['CLMT', 'F', 'LOB', 'TER']
sym_freq_3: ['AOS', 'BVH', 'CHRD', 'DNOW', 'ERF', 'FLR', 'IGT']
sym_freq_2: ['CAR', 'IRTC']

sym_freq_6: ['ASC', 'CVE', 'MRK', 'VLO']
sym_freq_5: ['ARLP', 'AXGN', 'CC', 'HRB', 'OEC', 'SQM']
sym_freq_4: ['AZPN', 'DK', 'NRG', 'STKL', 'SU', 'THS']
sym_freq_3: ['AMBC']
sym_freq_2: ['DINO', 'STNG', 'SWX']

sym_freq_6: ['DQ', 'FOLD', 'QURE']
sym_freq_5: ['AXSM', 'CELH', 'CYRX', 'EDU', 'FDP', 'RARE']
sym_freq_4: ['RCKT', 'RGNX', 'STKL']
sym_freq_3: ['AGYS', 'AMN', 'CLW', 'CORT', 'CVBF', 'DG']
sym_freq_2: ['ELF', 'RFP']

sym_freq_6: ['CLMT', 'WIRE']
sym_freq_5: ['ANET', 'ERF', 'LC', 'MARA', 'SAIA', 'WOLF']
sym_freq_4: ['AZTA', 'ENPH', 'IGT', 'SM', 'XENE']
sym_freq_

In [299]:
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  print(f'sym_freq_6: {l_sym_freq_cnt[0]}')
  print(f'sym_freq_5: {l_sym_freq_cnt[1]}')
  print(f'sym_freq_4: {l_sym_freq_cnt[2]}')
  print(f'sym_freq_3: {l_sym_freq_cnt[3]}\n')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]

  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')

      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.1f}, {grp_retnStd_d_UI[1]  :>17,.1f}, {grp_retnStd_d_UI[2]  :>17,.1f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.1f}, {grp_CAGR_d_retnStd[1]:>17,.1f}, {grp_CAGR_d_retnStd[2]:>17,.1f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.1f}, {grp_CAGR_d_UI[1]     :>17,.1f}, {grp_CAGR_d_UI[2]     :>17,.1f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.1f}, {grp_CAGR[1]          :>17,.1f}, {grp_CAGR[2]          :>17,.1f}')

      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')


        # df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
        # print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
        df_eval_sym_freq_results_validate.loc[len(df_eval_sym_freq_results_validate)] = row_add_total       
        print(f'appended row_add to df_eval_sym_freq_results_validate:\n{row_add_total}\n')        
  
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 114 max_lookback_slice
max_lookback_slice: (165, 225, 229)
max lookback dates: 2022-01-06, 2022-04-04, 2022-04-08
df_eval dates (inclusive): 2022-04-04 - 2022-04-07
top_set_syms_n_freq: [('CUTR', 9), ('AMPH', 6), ('EQT', 6), ('GLNG', 6), ('LNTH', 6), ('BPT', 5), ('SBS', 5), ('CALM', 4), ('IPI', 4), ('OXY', 4), ('RGLD', 4), ('AR', 3), ('CIB', 3), ('GOGO', 3), ('IRTC', 3), ('MOS', 3), ('PTEN', 3), ('RGNX', 3), ('RRC', 3), ('SPLK', 3)]

sym_freq_6: ['AMPH', 'EQT', 'GLNG', 'LNTH']
sym_freq_5: ['BPT', 'SBS']
sym_freq_4: ['CALM', 'IPI', 'OXY', 'RGLD']
sym_freq_3: ['AR', 'CIB', 'GOGO', 'IRTC', 'MOS', 'PTEN', 'RGNX', 'RRC', 'SPLK']


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:               0.5,             -99.5,             -49.6,              -0.8
grp(retnStd/UI):   mean, std, mean/std:           2,109.4,           3,650.4,               0.6
grp(CAGR/retnStd): mean, std, mean/std:           4,687.9,           6,240.7,               0.8
grp(CAGR/UI):      me

In [300]:
# pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)
pickle_dump(df_eval_sym_freq_results_validate, path_data_dump, fp_df_eval_sym_freq_results_validate)

In [301]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [305]:
df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
df

,n_samples,days_lookbacks,days_eval,n_top_syms,syms_start,syms_end,sym_freq_cnt,grp(CAGR)_mean,grp(CAGR)_std,grp(CAGR)_mean/std,grp(CAGR/UI)_mean,grp(CAGR/UI)_std,grp(CAGR/UI)_mean/std,grp(CAGR/retnStd)_mean,grp(CAGR/retnStd)_std,grp(CAGR/retnStd)_mean/std,grp(retnStd/UI)_mean,grp(retnStd/UI)_std,grp(retnStd/UI)_mean/std,SPY_CAGR,SPY_CAGR/UI,SPY_CAGR/retnStd,SPY_retnStd/UI
0,114,"[15, 30, 60,...",6,20,0,10,6,-9.060648e-01,2.047110e-01,-4.426068,-1.417674e+01,4.385355e+00,-3.232745,-2.938283e+01,1.395072e+01,-2.106187,0.585949,0.367768,1.593256,-0.989629,-16.940888,-49.329849,0.343421
1,114,"[15, 30, 60,...",6,20,0,10,5,-9.986618e-01,1.071932e-03,-931.646279,-1.360339e+01,2.038776e+00,-6.672329,-3.645348e+01,3.577501e+00,-10.189647,0.378527,0.075756,4.996672,-0.989629,-16.940888,-49.329849,0.343421
2,114,"[15, 30, 60,...",6,20,0,10,4,2.179304e+08,3.082001e+08,0.707107,8.990426e+09,1.271438e+10,0.707107,2.237311e+09,3.164036e+09,0.707107,1.566096,1.734474,0.902923,-0.989629,-16.940888,-49.329849,0.343421
3,114,"[15, 30, 60,...",6,20,0,10,6,8.287973e-01,1.739211e+00,0.476536,7.084092e+01,1.093169e+02,0.648032,3.137418e+01,5.404370e+01,0.580533,1.653390,0.369722,4.471976,-0.253862,-20.007051,-17.895907,1.117968
4,114,"[15, 30, 60,...",6,20,0,10,5,-6.475457e-01,5.205846e-01,-1.243882,2.019110e+01,1.111662e+02,0.181630,-4.432001e+00,6.368368e+01,-0.069594,1.026004,0.471329,2.176834,-0.253862,-20.007051,-17.895907,1.117968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7736,114,"[15, 30, 60]",4,20,0,10,3,2.515795e-01,9.088074e-01,0.276824,7.959812e+01,1.354325e+02,0.587733,2.593509e+01,6.982297e+01,0.371441,1.456737,0.636621,2.288232,0.582292,215.803013,103.313562,2.088816
7737,114,"[15, 30, 60]",4,20,0,10,6,1.253527e+00,2.253527e+00,0.556251,1.403186e+02,1.474331e+02,0.951744,7.617936e+01,8.735223e+01,0.872094,1.198187,0.561423,2.134196,-0.790839,-43.873867,-58.900365,0.744883
7738,114,"[15, 30, 60]",4,20,0,10,5,-9.621618e-01,5.531982e-02,-17.392715,-2.968757e+01,2.375205e+01,-1.249895,-4.040100e+01,3.192099e+01,-1.265656,0.752041,0.107377,7.003733,-0.790839,-43.873867,-58.900365,0.744883
7739,114,"[15, 30, 60]",4,20,0,10,4,1.861955e+00,2.841425e+00,0.655289,1.584498e+02,2.303186e+02,0.687959,7.335951e+01,1.359073e+02,0.539776,1.349574,0.444532,3.035941,-0.790839,-43.873867,-58.900365,0.744883


In [306]:
# pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
        .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
              'grp(CAGR/UI)_mean':       ['mean', 'std',],
              'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
              'SPY_CAGR':                ['mean', 'std',],
              'SPY_CAGR/UI':             ['mean', 'std',],
              'SPY_CAGR/retnStd':        ['mean', 'std',],                           
              })

In [307]:
tbl

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                SPY_CAGR              SPY_CAGR/UI               SPY_CAGR/retnStd             
                                                 count          mean           std              mean           std                   mean           std      mean        std          mean           std             mean          std
days_lookbacks   days_eval sym_freq_cnt                                                                                                                                                                                               
[120]            4         3                       114  2.510347e+03  1.281171e+04     1.155115e+09   6.465253e+09     4.563643e+04        1.754027e+05  3.407992  11.314772  3.930179e+06  2.025826e+07       408.207572  1523.880722
                 5         3                       114  8.857833e+02  4.714046e+03     7.634669e+08   6.460086e+09     2.043980e+04        1.050860e+05  0.980229   4.751738  1.062275e+06  8.452350e+06       131.356220   611.275272
                 6         3                       114  1.169878e+02  4.739030e+02     1.024052e+07   3.936572e+07     3.021095e+03        1.153412e+04  0.991003   4.517483  2.407404e+03  1.870929e+04        97.512973   420.642091
[15, 30, 60, ... 4         3                        75  9.093982e+03  7.803903e+04     3.947304e+07   2.377823e+08     1.713919e+05        1.464595e+06  1.201747   4.947571  1.482453e+06  7.511899e+06       146.223841   612.961166
                           4                       114  7.837829e+10  8.368343e+11     1.567566e+17   1.673669e+18     9.747244e+11        1.040684e+13  3.215575  10.068122  3.096685e+06  1.724888e+07       360.154669  1362.842834
                           5                       114  3.346490e+06  3.514200e+07     6.687828e+12   7.028448e+13     1.916933e+08        2.029915e+09  3.215575  10.068122  3.096685e+06  1.724888e+07       360.154669  1362.842834
                           6                       110  1.435224e+07  1.064358e+08     2.870280e+13   2.128719e+14     3.610947e+08        2.904372e+09  3.357540  10.222752  3.209294e+06  1.755214e+07       378.956349  1383.872086
                 5         3                        65  2.618358e+02  1.199472e+03     5.078096e+07   2.209594e+08     5.881227e+03        2.147050e+04  0.422828   3.098555  3.727793e+05  3.003652e+06        52.006021   400.725874
                           4                       114  1.127142e+04  1.018056e+05     2.350159e+10   2.273741e+11     2.661842e+05        2.024071e+06  2.696800   9.535759  3.454008e+06  1.988572e+07       359.680068  1476.427335
                           5                       114  4.104695e+07  3.085218e+08     9.178325e+13   6.898752e+14     7.657260e+08        5.755433e+09  2.696800   9.535759  3.454008e+06  1.988572e+07       359.680068  1476.427335
                           6                       109  3.055671e+04  2.887784e+05     6.909153e+08   3.630255e+09     5.968770e+05        5.484165e+06  2.843123   9.728369  3.612450e+06  2.032662e+07       378.759427  1507.423253
                 6         3                        60  6.570278e+02  3.414039e+03     5.632828e+07   3.985658e+08     1.203400e+04        6.523013e+04  0.731179   3.967406  5.916086e+02  2.831895e+03        66.562319   298.952787
                           4                       114  1.927087e+06  2.040989e+07     1.194470e+10   8.748881e+10     2.001047e+07        2.095198e+08  1.390653   4.206467  3.265433e+03  2.147335e+04       133.298343   404.647907
                           5                       114  1.135722e+06  1.210746e+07     2.779330e+12   2.965730e+13     2.100906e+07        2.238875e+08  1.390653   4.206467  3.265433e+03  2.147335e+04       133.298343   404.647907
                           6                       111  1.158432e+04  1.122225e+05     1.602412e+08   9.552773e+0

In [308]:
tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# tbl

In [309]:
tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
tbl.head(20)

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                SPY_CAGR              SPY_CAGR/UI               SPY_CAGR/retnStd              dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std
                                                 count          mean           std              mean           std                   mean           std      mean        std          mean           std             mean          std                                                                                                                    
days_lookbacks   days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                   
[15, 30]         4         5                       104  2.454632e+24  2.503243e+25     4.599074e+26   4.690154e+27     4.803255e+24        4.898378e+25  2.574747   6.302925  2.418441e+06  8.783808e+06       448.241677  1019.834724  2.454632e+24     4.599074e+26     4.803255e+24              0.098058             0.275329            -0.177271    
[60]             4         3                       114  6.398038e+23  6.831235e+24     1.198756e+26   1.279921e+27     1.251976e+24        1.336745e+25  7.536041  22.871523  1.322809e+07  4.529667e+07      1307.275598  4387.109244  6.398038e+23     1.198756e+26     1.251976e+24              0.093659             0.292032            -0.198374    
[30]             4         3                       114  4.976252e+23  5.313183e+24     9.323659e+25   9.954943e+26     9.737593e+23        1.039690e+25  4.760345  15.112558  7.773749e+06  2.985833e+07       727.411200  2181.618106  4.976252e+23     9.323659e+25     9.737593e+23              0.093659             0.260354            -0.166696    
[15, 30]         5         5                       105  2.935294e+18  3.007782e+19     6.148805e+20   6.300650e+21     6.319247e+18        6.475301e+19  1.813687   6.763613  1.672576e+06  1.228035e+07       243.601503   968.427072  2.935294e+18     6.148805e+20     6.319247e+18              0.097590             0.136199            -0.038609    
[15, 30, 60]     4         6                       106  1.912365e+17  1.968901e+18     2.307473e+18   2.375668e+19     3.441431e+17        3.543170e+18  4.216902  16.063055  7.041599e+06  3.157538e+07       798.395861  2965.385879  1.912365e+17     2.307473e+18     3.441431e+17              0.097129             0.223009            -0.125880    
[15, 30]         4         3                       114  8.890821e+16  6.682656e+17     1.072793e+18   8.063263e+18     1.599964e+17        1.202589e+18  2.684350   6.544412  2.778538e+06  9.916124e+06       450.631010   999.468745  8.890821e+16     1.072793e+18     1.599964e+17              0.133047             0.280204            -0.147157    
[30, 60, 120]    4         5                       111  1.217906e+11  9.031857e+11     2.435811e+17   1.806371e+18     1.708456e+12        1.266972e+13  4.522059  13.442306  4.012496e+06  2.412577e+07       609.448190  2510.590092  1.217906e+11     2.435811e+17     1.708456e+12              0.134846             0.166316            -0.031470    
[15, 30, 60, ... 4         4                       114  7.837829e+10  8.368343e+11     1.567566e+17   1.673669e+18     9.747244e+11        1.040684e+13  3.215575  10.068122  3.096685e+06  1.724888e+07       360.154669  1362.842834  7.837829e+10     1.567566e+17     9.747244e+11              0.093660             0.179530            -0.085869    
[15, 30]         4         4                       107  5.012622e+10  5.182621e+11     1.002524e+17   1.036524e+18     6.244416e+11        6.444988e+12  2.846915   6.717126  2.960309e+06  1.021175e+07       480.848517  1024

In [311]:
tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
tbl.head(30)

grp(CAGR)_mean                             grp(CAGR/UI)_mean               grp(CAGR/retnStd)_mean                SPY_CAGR              SPY_CAGR/UI               SPY_CAGR/retnStd              dif_CAGR_mean dif_CAGR/UI_mean dif_CAGR/retnStd_mean grp_CAGR/UI_mean/std SPY_CAGR/UI_mean/std dif_CAGR/UI_mean/std
                                                 count          mean           std              mean           std                   mean           std      mean        std          mean           std             mean          std                                                                                                                    
days_lookbacks   days_eval sym_freq_cnt                                                                                                                                                                                                                                                                                                                   
[120]            6         3                       114  1.169878e+02  4.739030e+02     1.024052e+07   3.936572e+07     3.021095e+03        1.153412e+04  0.991003   4.517483  2.407404e+03  1.870929e+04        97.512973   420.642091  1.159968e+02     1.023811e+07     2.923582e+03              0.260138             0.128674             0.131464    
[15, 30, 60, ... 5         3                        65  2.618358e+02  1.199472e+03     5.078096e+07   2.209594e+08     5.881227e+03        2.147050e+04  0.422828   3.098555  3.727793e+05  3.003652e+06        52.006021   400.725874  2.614130e+02     5.040818e+07     5.829221e+03              0.229820             0.124109             0.105712    
[60]             6         3                       114  1.760899e+04  1.848282e+05     3.924770e+07   1.922269e+08     2.697258e+05        2.805724e+06  0.451441   2.169756  9.694607e+04  7.305426e+05        64.831364   236.251096  1.760854e+04     3.915075e+07     2.696610e+05              0.204174             0.132704             0.071470    
[60, 120]        6         5                       105  5.284260e+01  3.315004e+02     1.375412e+07   7.149830e+07     1.537399e+03        7.549144e+03  1.264082   3.611477  2.810695e+03  1.969965e+04       145.130803   406.361200  5.157851e+01     1.375131e+07     1.392268e+03              0.192370             0.142677             0.049693    
[15, 30, 60]     4         3                       113  8.370809e+03  6.377631e+04     2.465608e+09   9.418088e+09     1.807108e+05        1.201675e+06  4.013736  15.577586  6.656980e+06  3.061188e+07       773.190655  2876.787777  8.366795e+03     2.458951e+09     1.799376e+05              0.261795             0.217464             0.044331    
[30, 60, 120]    5         6                        97  7.475722e+03  3.857668e+04     1.101221e+09   7.109085e+09     1.269130e+05        5.963416e+05  1.951284   6.565162  1.584444e+06  1.271277e+07       263.664071   994.442857  7.473771e+03     1.099637e+09     1.266493e+05              0.154903             0.124634             0.030269    
[15, 30, 60, ... 6         6                       111  1.158432e+04  1.122225e+05     1.602412e+08   9.552773e+08     1.982486e+05        1.933373e+06  1.448681   4.248258  3.354951e+03  2.175713e+04       138.979687   408.595075  1.158287e+04     1.602379e+08     1.981096e+05              0.167743             0.154200             0.013543    
                 5         6                       109  3.055671e+04  2.887784e+05     6.909153e+08   3.630255e+09     5.968770e+05        5.484165e+06  2.843123   9.728369  3.612450e+06  2.032662e+07       378.759427  1507.423253  3.055387e+04     6.873029e+08     5.964982e+05              0.190321             0.177720             0.012601    
[60, 120]        6         6                        90  2.593389e+01  8.694045e+01     3.245498e+07   1.958095e+08     1.620109e+03        6.077012e+03  1.345787   3.778391  3.323745e+03  2.127781e+04       156.464182   431

In [ ]:
if verbose:
  # from IPython.display import display, HTML
  # from myUtils import symb_perf_stats_vectorized_v8
  df = df_eval.copy()
  df_SPY = df_SPY.copy()  
  _my_df = [df_eval, df_SPY]
  for _df in _my_df:
    # print(f'_df:\n{_df}')
    (
      symbols,
      period_yr,
      retn,
      DD,
      UI,
      MDD,
      retnMean,
      retnStd,
      retnStd_div_UI,
      CAGR,
      CAGR_div_retnStd,
      CAGR_div_UI,
      grp_retnStd_div_UI,     
      grp_CAGR,
      grp_CAGR_div_retnStd,
      grp_CAGR_div_UI,
    ) = symb_perf_stats_vectorized_v8(_df)

    # display(HTML(_df.to_html()))
    print(f'_df:\n{_df}\n')
    print(f'symbols:\n{symbols}\n')
    print(f'period_yr:\n{period_yr}\n')
    print(f'retn:\n{retn}\n')
    print(f'DD:\n{DD}\n')
    print(f'UI:\n{UI}\n')
    print(f'MDD:\n{MDD}\n')
    print(f'retnMean:\n{retnMean}\n')
    print(f'retnStd:\n{retnStd}\n')
    print(f'retnStd_div_UI:\n{retnStd_div_UI}\n')
    print(f'CAGR:\n{CAGR}\n')
    print(f'CAGR_div_retnStd:\n{CAGR_div_retnStd}\n')
    print(f'CAGR_div_UI:\n{CAGR_div_UI}\n')
    print(f'grp_retnStd_div_UI:\n{grp_retnStd_div_UI}\n')
    print(f'grp_CAGR:\n{grp_CAGR}\n')
    print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')
    print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')            

In [ ]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [ ]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [ ]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement